<a href="https://colab.research.google.com/github/GavinButts/Fall-Data-Challenge-2022/blob/main/Code/Kevin/Ordinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
!pip install --upgrade --no-deps statsmodels
import scipy.stats as stats


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
sheet_id = "11rno0SjsOyV1JlevH3q40UMm1s5e6m7g"
sheet_name = "curated_2019-required"

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(url)


In [4]:
#create FOYESCNT variable that determines the amount of times each person said 'Yes' on the FO#### questions (ITEMS 72-74)
#Legend:  Number of 'Yes'

FOYESCNT = []      #variable that will be added to csv
yesCount = 0
for i in range(15500):
  if df['FOSTORY2X'][i] == 1:
    yesCount += 1
  if df['FOCRAFTS'][i] == 1:
    yesCount += 1
  if df['FOGAMES'][i] == 1:
    yesCount += 1
  if df['FOBUILDX'][i] == 1:
    yesCount += 1
  if df['FOSPORT'][i] == 1:
    yesCount += 1
  if df['FORESPON'][i] == 1:
    yesCount += 1
  if df['FOHISTX'][i] == 1:
    yesCount += 1
  if df['FOLIBRAYX'][i] == 1:
    yesCount += 1
  if df['FOBOOKSTX'][i] == 1:
    yesCount += 1
  if df['FODINNERX'][i] != 0:
    yesCount += 1
  FOYESCNT.append(yesCount)
  yesCount = 0

df["FOYESCNT"] = FOYESCNT
df.to_csv("sample.csv", index=False)  #adds variable to csv

In [5]:
#Data Pre-Processing
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=[0,1,2,3,4,5,6,7,8,9,10], ordered=True)
df["FOYESCNT"] = df["FOYESCNT"].astype(cat_type)
df["FOYESCNT"].dtype

CategoricalDtype(categories=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], ordered=True)

In [8]:

from statsmodels.miscmodels.ordinal_model import OrderedModel
df

,BASMID,ALLGRADEX,EDCPUB,SCCHOICE,SPUBCHOIX,SCONSIDR,SCHLHRSWK,EINTNET,MOSTIMPT,INTNUM,...,DSBLTY,HHPARN19X,HHPARN19_BRD,NUMSIBSX,PARGRADEX,RACEETH,INTACC,CENREG,ZIPLOCL,FOYESCNT
0,20191000012,12,1,2,3,2,4,4,-1,-1,...,2,1,1,1,3,4,1,4,11,6
1,20191000029,4,1,2,3,2,4,3,-1,2,...,2,2,2,1,3,3,1,4,12,4
2,20191000059,11,2,2,2,1,4,3,2,2,...,2,2,2,1,5,2,3,2,11,6
3,20191000070,6,1,1,1,1,4,4,-1,-1,...,2,2,2,1,2,3,1,2,31,10
4,20191000078,9,1,2,3,2,4,4,-1,-1,...,2,1,1,1,5,5,1,2,21,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15495,20191225472,7,1,2,2,2,4,4,-1,-1,...,2,1,1,2,4,4,1,3,23,5
15496,20191225475,2,1,2,2,2,4,4,-1,-1,...,2,1,1,3,5,2,1,2,22,7
15497,20191225477,12,1,1,3,2,4,4,-1,-1,...,2,1,1,2,2,3,1,1,13,7
15498,20191225479,13,1,2,2,1,4,4,-1,-1,...,1,2,2,0,4,1,3,4,11,8


In [10]:
mod_prob = OrderedModel(df['FOYESCNT'],
                        df[['EDCPUB', 'SEENJOY', 'SEGRADES','SEABSNT','FSNOTESX','FSMEMO','HDHEALTH','CSEX','CSPEAKX','P1MRSTA','P1EMPL','P1AGE',
                            'P2GUARD','TTLHHINC','DSBLTY','SEFUTUREX','NUMSIBSX','PARGRADEX','CENREG']],
                        distr='logit')
 
res_log = mod_prob.fit(method='bfgs')
res_log.summary()


Optimization terminated successfully.
         Current function value: 2.140512
         Iterations: 69
         Function evaluations: 73
         Gradient evaluations: 73


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:               FOYESCNT   Log-Likelihood:                -33178.
Model:                   OrderedModel   AIC:                         6.641e+04
Method:            Maximum Likelihood   BIC:                         6.664e+04
Date:                Fri, 21 Oct 2022                                         
Time:                        18:57:39                                         
No. Observations:               15500                                         
Df Residuals:                   15471                                         
Df Model:                          29                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
EDCPUB         0.0219      0.046      0.480      0.631      -0.067       0.111
SEENJOY       -0.4370      0.022    -20.033      0.000      -0.480      -0.394
SEGRADES       0.0965      0.011      8.749      0.000       0.075       0.118
SEABSNT       -0.0818      0.027     -3.027      0.002      -0.135      -0.029
FSNOTESX      -0.4218      0.031    -13.674      0.000      -0.482      -0.361
FSMEMO        -0.1729      0.049     -3.553      0.000      -0.268      -0.078
HDHEALTH      -0.1994      0.020     -9.823      0.000      -0.239      -0.160
CSEX           0.1529      0.029      5.330      0.000       0.097       0.209
CSPEAKX        0.0521      0.017      3.145      0.002       0.020       0.084
P1MRSTA       -0.0215      0.015     -1.433      0.152      -0.051       0.008
P1EMPL         0.0622      0.009      6.978      0.000       0.045       0.080
P1AGE         -0.0383      0.002    -21.600      0.000      -0.042      -0.035
P2GUARD       -0.0752      0.046     -1.653      0.098      -0.164       0.014
TTLHHINC      -0.0473      0.006     -7.443      0.000      -0.060      -0.035
DSBLTY        -0.1603      0.035     -4.532      0.000      -0.230      -0.091
SEFUTUREX      0.1857      0.014     13.405      0.000       0.159       0.213
NUMSIBSX      -0.0277      0.015     -1.837      0.066      -0.057       0.002
PARGRADEX      0.1088      0.016      6.804      0.000       0.077       0.140
CENREG         0.0189      0.014      1.377      0.169      -0.008       0.046
0/1           -7.5428      0.209    -36.070      0.000      -7.953      -7.133
1/2            0.4968      0.052      9.601      0.000       0.395       0.598
2/3            0.0439      0.034      1.302      0.193      -0.022       0.110
3/4           -0.1920      0.026     -7.392      0.000      -0.243      -0.141
4/5           -0.2789      0.021    -13.076      0.000      -0.321      -0.237
5/6           -0.3028      0.019    -15.999      0.000      -0.340      -0.266
6/7           -0.3016      0.018    -16.338      0.000      -0.338      -0.265
7/8           -0.2769      0.020    -13.893      0.000      -0.316      -0.238
8/9           -0.1453      0.023     -6.279      0.000      -0.191      -0.100
9/10           0.1555      0.030      5.232      0.000       0.097       0.214
==============================================================================
"""